# Human-in-the-Loop Middleware

Ref: https://docs.langchain.com/oss/python/langchain/middleware/built-in

This notebook demonstrates `HumanInTheLoopMiddleware`, which pauses agent
execution for human approval before executing specific tool calls.


## Setup

Configure `.env` before running. See `.env.sample`.


In [64]:
import rich
from dotenv import load_dotenv

load_dotenv()

True

## HumanInTheLoopMiddleware

Built-in middleware that requires human approval for specific tool calls:

**How it works:**

1. `after_model` hook checks if AIMessage contains tool calls
2. For tools in `interrupt_on`, calls `interrupt()` to pause execution
3. Human provides decision: `approve`, `edit`, or `reject`
4. On `reject`, creates ToolMessage with error status

**Requires checkpointer** - state must be persisted across interruptions.

**Configuration:**

- `True`: Allow all decisions (approve/edit/reject)
- `False`: Auto-approve (no interruption)
- `InterruptOnConfig`: Custom allowed decisions and description

**Note:** `edit` decision allows modifying tool arguments before execution (not covered here).


In [65]:
from datetime import datetime
from typing import Any

from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.agents.middleware.types import AgentMiddleware, AgentState
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.state import CompiledStateGraph
from langgraph.runtime import Runtime
from langgraph.types import Command, interrupt


@tool
def get_current_time() -> str:
    """Get the current time."""
    return datetime.now().isoformat()


@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to the specified recipient."""
    return f"Email sent to {to} with subject '{subject}'"


model = ChatAnthropic(model="claude-sonnet-4-5-20250929")
checkpointer = InMemorySaver()

# Require approval for send_email, auto-approve get_current_time
hitl = HumanInTheLoopMiddleware(
    interrupt_on={
        "send_email": {
            "allowed_decisions": ["approve", "reject"],
            "description": "Review email before sending",
        },
        "get_current_time": False,  # Auto-approve
    }
)

agent: CompiledStateGraph[Any] = create_agent(
    model=model,
    tools=[get_current_time, send_email],
    system_prompt="You are a helpful assistant.",
    checkpointer=checkpointer,
    middleware=[hitl],
)

## Basic Approval Flow

When agent tries to send an email, execution pauses for approval.


In [66]:
config: RunnableConfig = {"configurable": {"thread_id": "hitl-test-1"}}

# This will trigger an interrupt when agent tries to send email
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Send an email to bob@example.com saying hello"}]},
    config,
)

rich.print("result =", result)

result =
{
    'messages': [
        HumanMessage(
            content='Send an email to bob@example.com saying hello',
            additional_kwargs={},
            response_metadata={},
            id='58082682-203b-49f0-b7e6-3f3232df5317'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01ACdwBF9pwZe5MfH1Tsc5eg',
                    'input': {
                        'to': 'bob@example.com',
                        'subject': 'Hello',
                        'body': 'Hello,\n\nI hope this message finds you well.\n\nBest regards'
                    },
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01QYSMeqMmprEKG829UZUWNH',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 642,
                    'output_tokens': 104,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--ad034f16-48d1-4940-91e1-e2e5cf06863b-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {
                        'to': 'bob@example.com',
                        'subject': 'Hello',
                        'body': 'Hello,\n\nI hope this message finds you well.\n\nBest regards'
                    },
                    'id': 'toolu_01ACdwBF9pwZe5MfH1Tsc5eg',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 642,
                'output_tokens': 104,
                'total_tokens': 746,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        )
    ],
    '__interrupt__': [
        Interrupt(
            value={
                'action_requests': [
                    {
                        'name': 'send_email',
                        'args': {
                            'to': 'bob@example.com',
                            'subject': 'Hello',
                            'body': 'Hello,\n\nI hope this message finds you well.\n\nBest regards'
                        },
                        'description': 'Review email before sending'
                    }
                ],
                'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'reject']}]
            },
            id='e9317ce35cf7e8c2b5f329d750f90178'
        )
    ]
}

In [67]:
# Check the interrupt state
state = agent.get_state(config)
rich.print("next =", state.next)
rich.print("tasks =", state.tasks)

# Verify we're in an interrupted state
if state.next:
    rich.print("[bold yellow]Agent is paused, waiting for approval...[/bold yellow]")

next =
('HumanInTheLoopMiddleware.after_model',)

tasks =
(
    PregelTask(
        id='7c6691b3-d3a9-21ec-be57-3324546c6a3d',
        name='HumanInTheLoopMiddleware.after_model',
        path=('__pregel_pull', 'HumanInTheLoopMiddleware.after_model'),
        error=None,
        interrupts=(
            Interrupt(
                value={
                    'action_requests': [
                        {
                            'name': 'send_email',
                            'args': {
                                'to': 'bob@example.com',
                                'subject': 'Hello',
                                'body': 'Hello,\n\nI hope this message finds you well.\n\nBest regards'
                            },
                            'description': 'Review email before sending'
                        }
                    ],
                    'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'reject']}]
                },
                id='e9317ce35cf7e8c2b5f329d750f90178'
            ),
        ),
        state=None,
        result=None
    ),
)

Agent is paused, waiting for approval...

In [68]:
# Approve the email (only if interrupted)
state = agent.get_state(config)
if state.next:
    result = agent.invoke(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config,
    )
    rich.print("result =", result)
else:
    rich.print("[bold red]Not in interrupted state[/bold red]")

result =
{
    'messages': [
        HumanMessage(
            content='Send an email to bob@example.com saying hello',
            additional_kwargs={},
            response_metadata={},
            id='58082682-203b-49f0-b7e6-3f3232df5317'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01ACdwBF9pwZe5MfH1Tsc5eg',
                    'input': {
                        'to': 'bob@example.com',
                        'subject': 'Hello',
                        'body': 'Hello,\n\nI hope this message finds you well.\n\nBest regards'
                    },
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01QYSMeqMmprEKG829UZUWNH',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 642,
                    'output_tokens': 104,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--ad034f16-48d1-4940-91e1-e2e5cf06863b-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {
                        'to': 'bob@example.com',
                        'subject': 'Hello',
                        'body': 'Hello,\n\nI hope this message finds you well.\n\nBest regards'
                    },
                    'id': 'toolu_01ACdwBF9pwZe5MfH1Tsc5eg',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 642,
                'output_tokens': 104,
                'total_tokens': 746,
                'input_token_details': {
                    'cache_creation': 0,
                    'cache_read': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        ),
        ToolMessage(
            content="Email sent to bob@example.com with subject 'Hello'",
            name='send_email',
            id='c6801858-f9cf-4b5a-b9dc-7f6380017fbf',
            tool_call_id='toolu_01ACdwBF9pwZe5MfH1Tsc5eg'
        ),
        AIMessage(
            content='I\'ve successfully sent an email to bob@example.com with "Hello" as the subject and a friendly
greeting message in the body.',
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01Xkps63EcsJzJhrGHV9XN2e',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'end_turn',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 773,
                    'output_tokens': 31,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--18925c5f-da85-4403-8bb9-bf95013187c7-0',
            usage_metadata={
                'input_tokens': 773,
                'output_tokens': 31,
                'total_tokens': 804,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_

## Rejection Flow

User can reject a tool call and provide a reason.


In [69]:
config2: RunnableConfig = {"configurable": {"thread_id": "hitl-test-2"}}

# Request to send email with full details
message = "Send an email to carol@example.com with subject 'Meeting' and body 'See you at 10am'"
result = agent.invoke(
    {"messages": [{"role": "user", "content": message}]},
    config2,
)

rich.print("Paused for approval...")
rich.print("result =", result)

Paused for approval...

result =
{
    'messages': [
        HumanMessage(
            content="Send an email to carol@example.com with subject 'Meeting' and body 'See you at 10am'",
            additional_kwargs={},
            response_metadata={},
            id='d761b13a-43e2-4fe2-b9a1-d4be42145f9a'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01NQEyXLibz8KtK3LffD5Qbf',
                    'input': {'to': 'carol@example.com', 'subject': 'Meeting', 'body': 'See you at 10am'},
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01FYkCnnTGzYgjkbVQ7nuKgj',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 657,
                    'output_tokens': 96,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--8ede635a-d447-460f-a393-7b45b7b374f9-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {'to': 'carol@example.com', 'subject': 'Meeting', 'body': 'See you at 10am'},
                    'id': 'toolu_01NQEyXLibz8KtK3LffD5Qbf',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 657,
                'output_tokens': 96,
                'total_tokens': 753,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        )
    ],
    '__interrupt__': [
        Interrupt(
            value={
                'action_requests': [
                    {
                        'name': 'send_email',
                        'args': {'to': 'carol@example.com', 'subject': 'Meeting', 'body': 'See you at 10am'},
                        'description': 'Review email before sending'
                    }
                ],
                'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'reject']}]
            },
            id='3efcb192c1661a29757f4b639535a1d4'
        )
    ]
}

In [70]:
# On second thought, reject the email
state = agent.get_state(config2)
if state.next:
    result = agent.invoke(
        Command(resume={"decisions": [{"type": "reject", "message": "User rejected this action."}]}),
        config2,
    )
    rich.print("result =", result)
else:
    rich.print("[bold red]Not in interrupted state[/bold red]")

result =
{
    'messages': [
        HumanMessage(
            content="Send an email to carol@example.com with subject 'Meeting' and body 'See you at 10am'",
            additional_kwargs={},
            response_metadata={},
            id='d761b13a-43e2-4fe2-b9a1-d4be42145f9a'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01NQEyXLibz8KtK3LffD5Qbf',
                    'input': {'to': 'carol@example.com', 'subject': 'Meeting', 'body': 'See you at 10am'},
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01FYkCnnTGzYgjkbVQ7nuKgj',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 657,
                    'output_tokens': 96,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--8ede635a-d447-460f-a393-7b45b7b374f9-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {'to': 'carol@example.com', 'subject': 'Meeting', 'body': 'See you at 10am'},
                    'id': 'toolu_01NQEyXLibz8KtK3LffD5Qbf',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 657,
                'output_tokens': 96,
                'total_tokens': 753,
                'input_token_details': {
                    'cache_creation': 0,
                    'cache_read': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        ),
        ToolMessage(
            content='User rejected this action.',
            name='send_email',
            id='d2d902eb-bce6-4c62-bcb8-2bf70f6cc940',
            tool_call_id='toolu_01NQEyXLibz8KtK3LffD5Qbf',
            status='error'
        ),
        AIMessage(
            content='I attempted to send the email to carol@example.com with the subject "Meeting" and body "See 
you at 10am", but the action was rejected. This might be due to permission settings or a security restriction. 
\n\nWould you like me to try again, or would you prefer to send this email through a different method?',
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01HfejGgZqBLxPqUKeSTEa37',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'end_turn',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 778,
                    'output_tokens': 72,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--82334acb-9fde-4850-a203-df3983a16c3a-0',
            usage_metadata={
                'input_tokens': 778,
                'output_tokens': 72,
                'total_tokens': 850,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_

## Multiple Tool Calls

When the model requests multiple tool calls at once, you provide decisions for each.


In [71]:
# Create agent that requires approval for all send_email calls
checkpointer3 = InMemorySaver()
hitl_multi = HumanInTheLoopMiddleware(
    interrupt_on={
        "send_email": {"allowed_decisions": ["approve", "reject"]},
    }
)

agent_multi: CompiledStateGraph[Any] = create_agent(
    model=model,
    tools=[get_current_time, send_email],
    system_prompt="You are a helpful assistant. When asked to send multiple emails, call send_email for each.",
    checkpointer=checkpointer3,
    middleware=[hitl_multi],
)

config_multi: RunnableConfig = {"configurable": {"thread_id": "multi-test"}}

# Request multiple emails at once
message = "Send two emails: one to alice@example.com saying 'Hi', one to bob@example.com saying 'Hello'"
result = agent_multi.invoke(
    {"messages": [{"role": "user", "content": message}]},
    config_multi,
)
rich.print("result =", result)

result =
{
    'messages': [
        HumanMessage(
            content="Send two emails: one to alice@example.com saying 'Hi', one to bob@example.com saying 'Hello'",
            additional_kwargs={},
            response_metadata={},
            id='b7f02f04-1552-4e00-8005-b4fa7c0a9169'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01QS5FcfLX4SLtGjG71jQSsh',
                    'input': {'to': 'alice@example.com', 'subject': 'Hi', 'body': 'Hi'},
                    'name': 'send_email',
                    'type': 'tool_use'
                },
                {
                    'id': 'toolu_01QCs6vJxwNkhaUsBx7Lw3ui',
                    'input': {'to': 'bob@example.com', 'subject': 'Hello', 'body': 'Hello'},
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01C3fHLPo4xvqLHArkCfqhyW',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 673,
                    'output_tokens': 165,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--3502f127-7bd1-4adb-8841-3ff8e7a8d453-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {'to': 'alice@example.com', 'subject': 'Hi', 'body': 'Hi'},
                    'id': 'toolu_01QS5FcfLX4SLtGjG71jQSsh',
                    'type': 'tool_call'
                },
                {
                    'name': 'send_email',
                    'args': {'to': 'bob@example.com', 'subject': 'Hello', 'body': 'Hello'},
                    'id': 'toolu_01QCs6vJxwNkhaUsBx7Lw3ui',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 673,
                'output_tokens': 165,
                'total_tokens': 838,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        )
    ],
    '__interrupt__': [
        Interrupt(
            value={
                'action_requests': [
                    {
                        'name': 'send_email',
                        'args': {'to': 'alice@example.com', 'subject': 'Hi', 'body': 'Hi'},
                        'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'to': 
'alice@example.com', 'subject': 'Hi', 'body': 'Hi'}"
                    },
                    {
                        'name': 'send_email',
                        'args': {'to': 'bob@example.com', 'subject': 'Hello', 'body': 'Hello'},
                        'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'to': 
'bob@example.com', 'subject': 'Hello', 'body': 'Hello'}"
                    }
                ],
                'review_configs': [
                    {'action_name': 'send_email', 'allowed_decisions': ['approve', 'reject']},
                    {'action_name': 'send_email', 'allowed_decisions': ['approve', 'reject']}
                ]
            },
            id='81fc07adf45cc8cb53a68a480ec67397'
        )
    ]
}

In [72]:
# Check pending approvals
state = agent_multi.get_state(config_multi)
if state.next and state.tasks:
    # Show what's waiting for approval
    for task in state.tasks:
        if hasattr(task, "interrupts") and task.interrupts:
            for intr in task.interrupts:
                rich.print("Pending approval:", intr.value)

Pending approval:
{
    'action_requests': [
        {
            'name': 'send_email',
            'args': {'to': 'alice@example.com', 'subject': 'Hi', 'body': 'Hi'},
            'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'to': 'alice@example.com',
'subject': 'Hi', 'body': 'Hi'}"
        },
        {
            'name': 'send_email',
            'args': {'to': 'bob@example.com', 'subject': 'Hello', 'body': 'Hello'},
            'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'to': 'bob@example.com', 
'subject': 'Hello', 'body': 'Hello'}"
        }
    ],
    'review_configs': [
        {'action_name': 'send_email', 'allowed_decisions': ['approve', 'reject']},
        {'action_name': 'send_email', 'allowed_decisions': ['approve', 'reject']}
    ]
}

In [73]:
# Approve first email, reject second
state = agent_multi.get_state(config_multi)
if state.next:
    result = agent_multi.invoke(
        Command(
            resume={
                "decisions": [
                    {"type": "approve"},
                    {"type": "reject", "message": "User rejected this action."},
                ]
            }
        ),
        config_multi,
    )
    rich.print("result =", result)
else:
    rich.print("[bold red]Not in interrupted state[/bold red]")

result =
{
    'messages': [
        HumanMessage(
            content="Send two emails: one to alice@example.com saying 'Hi', one to bob@example.com saying 'Hello'",
            additional_kwargs={},
            response_metadata={},
            id='b7f02f04-1552-4e00-8005-b4fa7c0a9169'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01QS5FcfLX4SLtGjG71jQSsh',
                    'input': {'to': 'alice@example.com', 'subject': 'Hi', 'body': 'Hi'},
                    'name': 'send_email',
                    'type': 'tool_use'
                },
                {
                    'id': 'toolu_01QCs6vJxwNkhaUsBx7Lw3ui',
                    'input': {'to': 'bob@example.com', 'subject': 'Hello', 'body': 'Hello'},
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01C3fHLPo4xvqLHArkCfqhyW',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 673,
                    'output_tokens': 165,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--3502f127-7bd1-4adb-8841-3ff8e7a8d453-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {'to': 'alice@example.com', 'subject': 'Hi', 'body': 'Hi'},
                    'id': 'toolu_01QS5FcfLX4SLtGjG71jQSsh',
                    'type': 'tool_call'
                },
                {
                    'name': 'send_email',
                    'args': {'to': 'bob@example.com', 'subject': 'Hello', 'body': 'Hello'},
                    'id': 'toolu_01QCs6vJxwNkhaUsBx7Lw3ui',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 673,
                'output_tokens': 165,
                'total_tokens': 838,
                'input_token_details': {
                    'cache_creation': 0,
                    'cache_read': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        ),
        ToolMessage(
            content='User rejected this action.',
            name='send_email',
            id='7005ffaf-9713-4cc3-8ed2-ee5abbec539b',
            tool_call_id='toolu_01QCs6vJxwNkhaUsBx7Lw3ui',
            status='error'
        ),
        ToolMessage(
            content="Email sent to alice@example.com with subject 'Hi'",
            name='send_email',
            id='4f6edbe4-4d20-40cf-ac37-56db1fc8879f',
            tool_call_id='toolu_01QS5FcfLX4SLtGjG71jQSsh'
        ),
        AIMessage(
            content='I\'ve sent the email to alice@example.com with the message "Hi". However, the email to 
bob@example.com saying "Hello" was not sent because the action was rejected.',
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01K94NLZgupjPgovoSBsVecU',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'end_turn',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 920,
         

## Auto-Approve with Memory

Custom middleware that auto-approves previously approved tool calls.
This demonstrates combining HITL with custom logic.

**NOTE:** This example stores approved calls in memory (`self.approved_calls`).
In production, you would persist this to a database or use checkpointer state.


In [74]:
class AutoApproveMiddleware(AgentMiddleware[AgentState[Any], None]):
    """Middleware that remembers approved tool calls and auto-approves them."""

    def __init__(self, tools_requiring_approval: set[str]) -> None:
        self.tools_requiring_approval = tools_requiring_approval
        self.approved_calls: set[tuple[str, str]] = set()  # (tool_name, args_hash)

    def _hash_args(self, args: dict[str, Any]) -> str:
        """Create a hashable representation of tool args."""
        return str(sorted(args.items()))

    def after_model(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        messages = state["messages"]
        if not messages:
            return None

        last_ai_msg = next((msg for msg in reversed(messages) if isinstance(msg, AIMessage)), None)
        if not last_ai_msg or not last_ai_msg.tool_calls:
            return None

        for tool_call in last_ai_msg.tool_calls:
            tool_name = tool_call["name"]
            if tool_name not in self.tools_requiring_approval:
                continue

            args_hash = self._hash_args(tool_call["args"])
            call_key = (tool_name, args_hash)

            if call_key in self.approved_calls:
                rich.print(f"[bold green]Auto-approved: {tool_name}[/bold green]")
                continue

            # Request approval
            rich.print(f"[bold yellow]Requesting approval for: {tool_name}[/bold yellow]")
            rich.print(f"Args: {tool_call['args']}")

            decision = interrupt({"tool": tool_name, "args": tool_call["args"]})

            if decision.get("approved"):
                self.approved_calls.add(call_key)
                rich.print(f"[bold green]Approved and remembered: {tool_name}[/bold green]")
            else:
                rich.print(f"[bold red]Rejected: {tool_name}[/bold red]")
                return {
                    "messages": [
                        ToolMessage(
                            content=f"Tool call rejected: {decision.get('reason', 'No reason provided')}",
                            tool_call_id=tool_call["id"],
                            status="error",
                        )
                    ]
                }

        return None

In [75]:
checkpointer2 = InMemorySaver()
auto_approve = AutoApproveMiddleware(tools_requiring_approval={"send_email"})

agent2: CompiledStateGraph[Any] = create_agent(
    model=model,
    tools=[get_current_time, send_email],
    system_prompt="You are a helpful assistant.",
    checkpointer=checkpointer2,
    middleware=[auto_approve],
)

config3: RunnableConfig = {"configurable": {"thread_id": "auto-approve-test"}}

In [76]:
# First call - will request approval
result = agent2.invoke(
    {"messages": [{"role": "user", "content": "Send email to alice@example.com saying 'See you tomorrow'"}]},
    config3,
)
rich.print("result =", result)

Requesting approval for: send_email

Args: {'to': 'alice@example.com', 'subject': 'See you tomorrow', 'body': 'See you tomorrow'}

result =
{
    'messages': [
        HumanMessage(
            content="Send email to alice@example.com saying 'See you tomorrow'",
            additional_kwargs={},
            response_metadata={},
            id='fc4e6060-9eac-46c1-baa7-a9662e5bb72c'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_012r1McnS25ohnNV7J2FAUmg',
                    'input': {
                        'to': 'alice@example.com',
                        'subject': 'See you tomorrow',
                        'body': 'See you tomorrow'
                    },
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01KYCWtCr4npkBHMNjeNTDGf',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 646,
                    'output_tokens': 95,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--4891e38b-fe9a-4174-9f05-ea6f25210fa2-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {
                        'to': 'alice@example.com',
                        'subject': 'See you tomorrow',
                        'body': 'See you tomorrow'
                    },
                    'id': 'toolu_012r1McnS25ohnNV7J2FAUmg',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 646,
                'output_tokens': 95,
                'total_tokens': 741,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        )
    ],
    '__interrupt__': [
        Interrupt(
            value={
                'tool': 'send_email',
                'args': {'to': 'alice@example.com', 'subject': 'See you tomorrow', 'body': 'See you tomorrow'}
            },
            id='951f0f70fe561035467114ef3d3f4b48'
        )
    ]
}

In [77]:
# Approve (only if interrupted)
state = agent2.get_state(config3)
if state.next:
    result = agent2.invoke(
        Command(resume={"approved": True}),
        config3,
    )
    rich.print("result =", result)
else:
    rich.print("[bold red]Not in interrupted state[/bold red]")

Requesting approval for: send_email

Args: {'to': 'alice@example.com', 'subject': 'See you tomorrow', 'body': 'See you tomorrow'}

Approved and remembered: send_email

result =
{
    'messages': [
        HumanMessage(
            content="Send email to alice@example.com saying 'See you tomorrow'",
            additional_kwargs={},
            response_metadata={},
            id='fc4e6060-9eac-46c1-baa7-a9662e5bb72c'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_012r1McnS25ohnNV7J2FAUmg',
                    'input': {
                        'to': 'alice@example.com',
                        'subject': 'See you tomorrow',
                        'body': 'See you tomorrow'
                    },
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01KYCWtCr4npkBHMNjeNTDGf',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 646,
                    'output_tokens': 95,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--4891e38b-fe9a-4174-9f05-ea6f25210fa2-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {
                        'to': 'alice@example.com',
                        'subject': 'See you tomorrow',
                        'body': 'See you tomorrow'
                    },
                    'id': 'toolu_012r1McnS25ohnNV7J2FAUmg',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 646,
                'output_tokens': 95,
                'total_tokens': 741,
                'input_token_details': {
                    'cache_creation': 0,
                    'cache_read': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        ),
        ToolMessage(
            content="Email sent to alice@example.com with subject 'See you tomorrow'",
            name='send_email',
            id='1a5a226c-b8cd-44d6-8110-9ec82c563d92',
            tool_call_id='toolu_012r1McnS25ohnNV7J2FAUmg'
        ),
        AIMessage(
            content='I\'ve successfully sent the email to alice@example.com with the message "See you tomorrow" as 
both the subject and body of the email.',
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01E16KPaGbVPxsrvHmo5cnVm',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'end_turn',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 770,
                    'output_tokens': 33,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--9327f4bb-2f27-4a5d-b4ce-48c88dbd7a83-0',
            usage_metadata={
                'input_tokens': 770,
                'output_tokens': 33,
                'total_tokens': 803,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                 

In [78]:
# Same call again - should be auto-approved
config4: RunnableConfig = {"configurable": {"thread_id": "auto-approve-test-2"}}

result = agent2.invoke(
    {"messages": [{"role": "user", "content": "Send email to alice@example.com saying 'See you tomorrow'"}]},
    config4,
)
rich.print("result =", result)

Auto-approved: send_email

result =
{
    'messages': [
        HumanMessage(
            content="Send email to alice@example.com saying 'See you tomorrow'",
            additional_kwargs={},
            response_metadata={},
            id='dcac3cb8-c9a8-41a4-916c-9578aee3e37c'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01Y9oqSCkXbeHaZjQwn8uP4n',
                    'input': {
                        'to': 'alice@example.com',
                        'subject': 'See you tomorrow',
                        'body': 'See you tomorrow'
                    },
                    'name': 'send_email',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01NK6Fnsj4FPq5z85qSGDpHR',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 646,
                    'output_tokens': 95,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--5dd8b48a-6712-4c1d-ab22-ffb864adef2c-0',
            tool_calls=[
                {
                    'name': 'send_email',
                    'args': {
                        'to': 'alice@example.com',
                        'subject': 'See you tomorrow',
                        'body': 'See you tomorrow'
                    },
                    'id': 'toolu_01Y9oqSCkXbeHaZjQwn8uP4n',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 646,
                'output_tokens': 95,
                'total_tokens': 741,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        ),
        ToolMessage(
            content="Email sent to alice@example.com with subject 'See you tomorrow'",
            name='send_email',
            id='11f52ef3-f83b-4629-865f-de0395ed89b0',
            tool_call_id='toolu_01Y9oqSCkXbeHaZjQwn8uP4n'
        ),
        AIMessage(
            content='I\'ve successfully sent the email to alice@example.com with the message "See you tomorrow" as 
both the subject and body.',
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01Ehe53MeZdCMqTd9vKRSivm',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'end_turn',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 770,
                    'output_tokens': 30,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--9243757e-eecf-4e4d-95fe-9b0f02255860-0',
            usage_metadata={
                'input_tokens': 770,
                'output_tokens': 30,
                'total_tokens': 800,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral